## Google Colab demo (3–5 min)

Этот ноутбук показывает:

- где лежат **датасеты** (`data/*.csv`, `data/*.json`)
- какие **скрипты** генерируют сценарии с “внедрёнными” проблемами (`generate_scenarios.py`) и запускают проверки (`main.py`, `src/frameworks/*_adapter.py`)
- **команду запуска** и где формируются **отчёты/таблицы** (`reports/<run_name>/<timestamp>/...`)

Репозиторий: `https://github.com/alexandor09/ml-testing-frameworks-comparison/`



In [ ]:
import os

REPO_URL = "https://github.com/alexandor09/ml-testing-frameworks-comparison.git"
REPO_DIR = "ml-testing-frameworks-comparison"

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL}

%cd {REPO_DIR}
!python --version



In [ ]:
# Варианты:
# - "light": ставим минимум для запуска одного фреймворка (по умолчанию GX)
# - "full": ставим всё из requirements.txt (может быть заметно дольше)
import sys
import subprocess

INSTALL_MODE = "light"  # "light" | "full"

if INSTALL_MODE == "full":
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-r", "requirements.txt"])
else:
    # main.py теперь импортирует адаптеры лениво, поэтому можно не ставить весь стек.
    subprocess.check_call([
        sys.executable, "-m", "pip", "install", "-q",
        "pandas", "numpy", "prophet", "great_expectations", "psutil", "plotly", "scikit-learn", "tqdm",
    ])



In [ ]:
from pathlib import Path

print("### data/")
data_dir = Path("data")
for p in sorted(data_dir.glob("*.csv")):
    print("-", p.as_posix())



### Где считаются “внедрённые” и “выявленные” проблемы

- **Внедрённые** (ground truth): в `generate_scenarios.py`.
  - `pass`: пропуски (`price` ~5%, `y` ~2%) + дубликаты (~3%).
  - `dr`: дрейф в **тестовом хвосте** (последние 20%) + выбросы в `y` и `price`.
- **Выявленные**: в `src/frameworks/*_adapter.py` через поля `issues_detected` и `check_values`.



In [ ]:
import sys
import subprocess

# Быстрый демо-запуск: один фреймворк (GX) на сценарии dr
INPUT = "data/dr.csv"
OUTPUT = "reports/colab_demo_dr"
FRAMEWORK = "gx"  # gx | evidently | alibi | nannyml

subprocess.check_call([
    sys.executable, "main.py",
    "--input", INPUT,
    "--format", "csv",
    "--output", OUTPUT,
    "--framework", FRAMEWORK,
])



In [ ]:
import json
from pathlib import Path

run_base = Path(OUTPUT)
runs = sorted([p for p in run_base.glob("*") if p.is_dir()])
latest = runs[-1]
print("Latest run:", latest.as_posix())

print("\n### files")
for p in sorted(latest.glob("*") ):
    print("-", p.name)

summary_path = latest / "comparison_summary.json"
print("\n### comparison_summary.json")
print(summary_path.as_posix())
print(json.dumps(json.loads(summary_path.read_text(encoding="utf-8")), indent=2, ensure_ascii=False)[:4000])



### (Опционально) полный прогон со всеми 4 фреймворками и HTML-дашбордом

Если хотите получить `dashboard.html` и консольную сравнительную таблицу как в локальном запуске — переключите `INSTALL_MODE = "full"` (выше) и выполните ячейку ниже.



In [ ]:
# FULL RUN (может быть дольше)
# - создаст dashboard.html и final_summary.json
# - все артефакты будут в reports/colab_demo_full/<timestamp>/

import sys
import subprocess
import json
from pathlib import Path

INPUT_FULL = "data/dr.csv"
OUTPUT_FULL = "reports/colab_demo_full"

subprocess.check_call([
    sys.executable, "main.py",
    "--input", INPUT_FULL,
    "--format", "csv",
    "--output", OUTPUT_FULL,
])

run_base = Path(OUTPUT_FULL)
runs = sorted([p for p in run_base.glob("*") if p.is_dir()])
latest = runs[-1]
print("Latest full run:", latest.as_posix())

print("\nDashboard:", (latest / "dashboard.html").as_posix())
print("Final summary:", (latest / "final_summary.json").as_posix())

# В Colab самый простой способ посмотреть HTML — скачать файл и открыть локально в браузере.
# from google.colab import files
# files.download((latest / "dashboard.html").as_posix())

